### TODO
- Now we can take the special cases we added to info_grupos back to frac_vars.xlsx since we have them in cols to avoid.
- We have two negative vars that are not -999 that we need to confirm that we can sample.

In [31]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
import pickle
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
from info_grupos import prefijos, grupos_normaliza, frac_vars_special_cases_list
from genera_muestra import GenerateLHCSample

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

In [32]:
# Record the start time
start_time = time.time()

In [33]:
## Define paths and global variables

FILE_PATH = os.getcwd()
build_path = lambda PATH : os.path.abspath(os.path.join(*PATH))

DATA_PATH = build_path([FILE_PATH, "..", "data"])
OUTPUT_PATH = build_path([FILE_PATH, "..", "output"])

SSP_OUTPUT_PATH = build_path([OUTPUT_PATH, "ssp"])

REAL_DATA_FILE_PATH = build_path([DATA_PATH, "real_data.csv"]) 

SALIDAS_EXPERIMENTOS_PATH = build_path([OUTPUT_PATH, "experiments"]) 

INPUTS_ESTRESADOS_PATH = build_path([SALIDAS_EXPERIMENTOS_PATH, "sim_inputs"])
OUTPUTS_ESTRESADOS_PATH = build_path([SALIDAS_EXPERIMENTOS_PATH, "sim_outputs"])

target_country = "croatia"

In [ ]:
# Funciones
def print_elapsed_time(start_time):

    # Record the end time
    end_time = time.time()

    # Calculate and print the execution time
    execution_time = end_time - start_time
    print(f"------------------------ EXECUTION TIME: {execution_time} seconds ------------------------")

def check_land_use_factor(ssp_object, target_country):
    dict_scendata = ssp_object.generate_scenario_database_from_primary_key(0)
    df_inputs_check = dict_scendata.get(target_country) # Change the name of the country if running a different one
    lndu_realloc_fact_df = ssp_object.model_attributes.extract_model_variable(df_inputs_check, "Land Use Yield Reallocation Factor")

    if lndu_realloc_fact_df['lndu_reallocation_factor'].sum() > 0:
        raise ValueError(" --------------- The sum of 'lndu_reallocation_factor' is greater than 0. Script terminated. -----------------")
    
# Some helpful functions to check differences between dataframes and fix them

def compare_dfs(df1, df2):
    # Assuming your DataFrames are df1 and df2
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)

    # Columns present in df1 but not in df2
    diff_in_df1 = columns_df1 - columns_df2

    # Columns present in df2 but not in df1
    diff_in_df2 = columns_df2 - columns_df1

    print("Columns in df1 but not in df2:", diff_in_df1)
    print("Columns in df2 but not in df1:", diff_in_df2)

def add_missing_cols(df1, df2):
    # Identify columns in df1 but not in df2
    columns_to_add = [col for col in df1.columns if col not in df2.columns]

    # Add missing columns to df2 with their values from df1
    for col in columns_to_add:
        df2[col] = df1[col]
    
    return df2

def get_indicators_col_names(df, cols_with_issue = []):

    cols_to_avoid = ['time_period', 'region'] + cols_with_issue
    col_names = [col for col in df.columns if col not in cols_to_avoid]

    # Check if the length of col_names is as expected
    expected_length = len(df.columns) - len(cols_to_avoid)
    print(f"Expected length after removal: {expected_length}")
    print(f"Actual length of col_names: {len(col_names)}")

    # Verify if all cols_to_avoid were removed from col_names
    removed_successfully = all(col not in col_names for col in cols_to_avoid)
    if removed_successfully:
        print("All columns in cols_to_avoid were successfully removed.")
    else:
        print("Some columns in cols_to_avoid are still present in col_names.")
        # Optionally, print the columns that were not removed
        remaining_cols = [col for col in cols_to_avoid if col in col_names]
        print("Columns not removed:", remaining_cols)

    return col_names

In [35]:
### Cargamos datos de ejemplo de costa rica

examples = SISEPUEDEExamples()
cr = examples("input_data_frame")

In [36]:
df_input = pd.read_csv(REAL_DATA_FILE_PATH)
df_input.head()

,region,iso_code3,period,area_gnrl_country_ha,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,...,yf_agrc_fruits_tonne_ha,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha
0,croatia,HRV,0,8807000,70,2923,31.751466,6468,315,508,...,5.546667,28.8742,0.602367,1.930675,0.5205,2.638183,0,0,35.76480,21.067738
1,croatia,HRV,1,8807000,70,2923,31.751466,6468,315,508,...,5.555383,29.6558,0.479900,2.304050,0.7342,3.256933,0,0,47.57660,20.412554
2,croatia,HRV,2,8807000,70,2923,31.751466,6468,315,508,...,4.304906,30.0390,0.360400,2.075950,0.8615,3.199083,0,0,41.09780,22.329531
3,croatia,HRV,3,8807000,70,2923,31.751466,6468,315,508,...,6.272229,30.0390,0.160300,2.662133,0.9852,3.266680,0,0,37.42445,21.505800
4,croatia,HRV,4,8807000,70,2923,31.751466,6468,315,508,...,5.878853,30.0390,0.196325,2.543567,0.9045,4.068040,0,0,39.81490,21.874247


In [37]:
df_input = df_input.rename(columns={'period':'time_period'})
df_input = add_missing_cols(cr, df_input.copy())
df_input = df_input.drop(columns='iso_code3')

In [38]:
df_input.head()

,region,time_period,area_gnrl_country_ha,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,avgmass_lvst_animal_cattle_nondairy_kg,...,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_electrolysis_water,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_electricity,energydensity_gravimetric_enfu_gj_per_tonne_fuel_ammonia,energydensity_gravimetric_enfu_gj_per_tonne_fuel_water,frac_trns_fuelmix_water_borne_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen,nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_natural_gas,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil
0,croatia,0,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
1,croatia,1,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
2,croatia,2,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
3,croatia,3,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
4,croatia,4,8807000,70,2923,31.751466,6468,315,508,303,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0


In [39]:
# Double checking that our df is in the correct shape MAKE SURE THIS IS OK THEY HAVE TO BE EQUAL!
compare_dfs(cr, df_input)

Columns in df1 but not in df2: set()
Columns in df2 but not in df1: set()


In [40]:
# Checking if there are any columns with null values in it
columns_with_na = df_input.columns[df_input.isna().any()].tolist()

print(columns_with_na)

[]


In [78]:
# Select only numeric columns
numeric_cols = df_input.select_dtypes(include=[np.number]).columns

# Identify columns with values lower than 0
columns_with_negatives = df_input[numeric_cols].columns[(df_input[numeric_cols] < 0).any()].tolist()
columns_with_negatives


['elasticity_gnrl_rate_occupancy_to_gdppc',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biogas',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biomass',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal_ccs',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas_ccs',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_geothermal',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_hydropower',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_nuclear',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_ocean',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_oil',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_solar',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_waste_incineration',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_p

In [77]:
columns_all_999 = df_input.columns[(df_input == -999).any()].tolist()
columns_all_999

['frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biogas',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biomass',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal_ccs',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas_ccs',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_geothermal',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_hydropower',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_nuclear',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_ocean',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_oil',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_solar',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_waste_incineration',
 'frac_entc_max_elec_production_increase_to_satisfy_msp_pp_wind',
 'limit_gnrl_annual_emissions_mt_ch

In [80]:
list(set(columns_with_negatives) - set(columns_all_999))

['elasticity_gnrl_rate_occupancy_to_gdppc', 'net_imports_cement_clinker_tonne']

In [81]:
df_input[['elasticity_gnrl_rate_occupancy_to_gdppc', 'net_imports_cement_clinker_tonne']].head()

,elasticity_gnrl_rate_occupancy_to_gdppc,net_imports_cement_clinker_tonne
0,-0.1,-350349.634
1,-0.1,-213653.110
2,-0.1,-496174.119
3,-0.1,-163715.780
4,-0.1,45353.610


In [ ]:
# Avoid land use stuff and some frac special cases
pij_cols = [col for col in df_input.columns if col.startswith('pij')]
cols_to_avoid = pij_cols + frac_vars_special_cases_list + columns_all_999
campos_estresar = get_indicators_col_names(df_input, cols_with_issue=cols_to_avoid)

Expected length after removal: 2047
Actual length of col_names: 2047
All columns in cols_to_avoid were successfully removed.


In [44]:
generate_sample = GenerateLHCSample(n=100, n_var=len(campos_estresar))
generate_sample.generate_sample()
with open('sample_scaled.pickle', 'rb') as handle:
    sample_scaled = pickle.load(handle)

In [45]:
id_experimento = 1
    
df_estresado = df_input.copy()
df_estresado[campos_estresar]  = (df_input[campos_estresar]*sample_scaled[id_experimento]).to_numpy()

In [46]:
# Load new groups that need normalization
df_frac_vars = pd.read_excel('frac_vars.xlsx', sheet_name='frac_vars_no_special_cases')
df_frac_vars.head()

,frac_var_name,frac_var_name_prefix
0,frac_agrc_bevs_and_spices_cl1_temperate,frac_agrc_bevs_and_spices_cl1
1,frac_agrc_bevs_and_spices_cl1_tropical,frac_agrc_bevs_and_spices_cl1
2,frac_agrc_bevs_and_spices_cl2_dry,frac_agrc_bevs_and_spices_cl2
3,frac_agrc_bevs_and_spices_cl2_wet,frac_agrc_bevs_and_spices_cl2
4,frac_agrc_cereals_cl1_temperate,frac_agrc_cereals_cl1


In [47]:
need_norm_prefix = df_frac_vars.frac_var_name_prefix.unique()

In [ ]:
### Normaliza grupo de variables para que sumen 1 en conjunto

for grupo in need_norm_prefix:
    vars_grupo = [i for i in df_estresado.columns if grupo in i]
    
    # Skip normalization for columns in cols_to_avoid
    if any(col in cols_to_avoid for col in vars_grupo):
        continue

    # Apply conditional log transformation
    df_estresado[vars_grupo] = df_estresado[vars_grupo].applymap(lambda y: -np.log(y) if y != 0 else 0)
    
    # Check if the sum is zero before normalizing
    sum_values = df_estresado[vars_grupo].sum(axis=1)
    df_estresado[vars_grupo] = df_estresado[vars_grupo].div(sum_values, axis=0).fillna(0)


# This is also an special case
ce_problematic = ['frac_waso_biogas_food',
                  'frac_waso_biogas_sludge',
                  'frac_waso_biogas_yard',
                  'frac_waso_compost_food',
                  'frac_waso_compost_methane_flared',
                  'frac_waso_compost_sludge',
                  'frac_waso_compost_yard']

# Apply conditional log transformation
df_estresado[ce_problematic] = df_estresado[ce_problematic].applymap(lambda y: -np.log(y) if y != 0 else 0)
# Check if the sum is zero before normalizing
sum_values = df_estresado[ce_problematic].sum(axis=1)
df_estresado[ce_problematic] = df_estresado[ce_problematic].div(sum_values, axis=0).fillna(0)



####################

In [56]:
# # Assuming df_estresado is defined and contains columns
# vars_grupo = [i for i in df_estresado.columns if i.startswith('frac_')]

# df_frac_vars = pd.DataFrame(vars_grupo, columns=['frac_var_name'])
# df_frac_vars.sort_values(by='frac_var_name', inplace=True)

# # Extract prefix by removing the last '_{word}' segment
# df_frac_vars['frac_var_name_prefix'] = df_frac_vars['frac_var_name'].apply(lambda x: '_'.join(x.split('_')[:-1]))

# df_frac_vars.to_csv('frac_vars.csv', index=False)

###########

In [57]:
# Checking if there are any columns with null values in it
columns_with_na = df_estresado.columns[df_estresado.isna().any()].tolist()
print(columns_with_na)
if columns_with_na:
    df_estresado[columns_with_na] = df_estresado[columns_with_na].fillna(0)

columns_with_na = df_estresado.columns[df_estresado.isna().any()].tolist()
print(columns_with_na)

[]
[]


In [58]:
transformers = trf.transformers.Transformers(
    {},
    df_input = df_estresado,
)

##  SETUP SOME SISEPUEDE STUFF

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)

# set an ouput path and instantiate

trf.instantiate_default_strategy_directory(
        transformers,
        SSP_OUTPUT_PATH,
    )

# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
        SSP_OUTPUT_PATH,
        transformers = transformers,
    )

strategies = trf.Strategies(
        transformations,
        export_path = "transformations",
        prebuild = True,
    )

In [59]:


# call the example
df_vargroups = examples("variable_trajectory_group_specification")

strategies.build_strategies_to_templates(
        df_trajgroup = df_vargroups,
        include_simplex_group_as_trajgroup = True,
        strategies = [0, 1000],
    )



0

In [60]:
import sisepuede as si
ssp = si.SISEPUEDE(
        "calibrated",
        initialize_as_dummy = False, # no connection to Julia is initialized if set to True
        regions = [target_country],
        db_type = "csv",
        strategies = strategies,
        try_exogenous_xl_types_in_variable_specification = True,
    )

2024-11-12 19:45:24,324 - INFO - Successfully initialized SISEPUEDEFileStructure.
2024-11-12 19:45:24,326 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2024-11-12 19:45:24,327 - INFO - 	Setting export engine to 'csv'.
2024-11-12 19:45:24,328 - WARNING - No index fields defined. Index field values will not be checked when writing to tables.
2024-11-12 19:45:24,329 - INFO - Successfully instantiated table ANALYSIS_METADATA
2024-11-12 19:45:24,330 - WARNING - No index fields found in ATTRIBUTE_DESIGN. Initializing index fields.
2024-11-12 19:45:24,330 - INFO - Successfully instantiated table ATTRIBUTE_DESIGN
2024-11-12 19:45:24,331 - WARNING - No index fields found in ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES. Initializing index fields.
2024-11-12 19:45:24,332 - INFO - Successfully instantiated table ATTRIBUTE_LHC_SAMPLES_EXOGENOUS_UNCERTAINTIES
2024-11-12 19:45:24,333 - WARNING - No index fi

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Precompiling NemoMod...
Info Given NemoMod was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
   2348.1 ms  ? NemoMod
[ Info: Precompiling NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72] 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
┌ Info: Skipping precompilation due to precompilable error. Importing NemoMod [a3c327a0-d2f0-11e8-37fd-d12fd35c3c72].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
2024-11-12 19:46:14,736 - INFO - Successfully initialized JuMP optimizer from solver module HiGHS.
2024-11-12 19:46:14,761 - INFO - Successfully initialized SISEPUEDEModels.
2024-11-12 19:46:14,769 - INFO - Table ANALYSIS_METADATA successfully written to /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11

In [61]:
# Checks if the land use reallocation factor is set to 0.0
check_land_use_factor(ssp_object=ssp, target_country=target_country)

In [62]:
# Create parameters dict for the model to run
dict_run = {
        ssp.key_future: [0],
        ssp.key_design: [0],
        ssp.key_strategy: [
            0,
            1000,
        ],
    }


In [63]:
# we'll save inputs since we're doing a small set of runs
ssp.project_scenarios(
        dict_run,
        save_inputs = True,
    )

2024-11-12 19:46:18,264 - INFO - 
***	STARTING REGION croatia	***

2024-11-12 19:46:21,514 - INFO - Trying run primary_id = 0 in region croatia
2024-11-12 19:46:21,515 - INFO - Running AFOLU model
2024-11-12 19:46:21,681 - INFO - AFOLU model run successfully completed
2024-11-12 19:46:21,682 - INFO - Running CircularEconomy model
2024-11-12 19:46:21,729 - INFO - CircularEconomy model run successfully completed
2024-11-12 19:46:21,730 - INFO - Running IPPU model
2024-11-12 19:46:21,807 - INFO - IPPU model run successfully completed
2024-11-12 19:46:21,808 - INFO - Running Energy model (EnergyConsumption without Fugitive Emissions)
2024-11-12 19:46:21,828 - DEBUG - Missing elasticity information found in 'project_energy_consumption_by_fuel_from_effvars': using specified future demands.
2024-11-12 19:46:21,918 - INFO - EnergyConsumption without Fugitive Emissions model run successfully completed
2024-11-12 19:46:21,919 - INFO - Running Energy model (Electricity and Fuel Production: trying

2024-12-Nov 19:46:22.428 Opened SQLite database at /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11/site-packages/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2024-12-Nov 19:46:22.751 Added NEMO structure to SQLite database at /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11/site-packages/sisepuede/tmp/nemomod_intermediate_database.sqlite.
2024-12-Nov 19:46:38.701 Started modeling scenario. NEMO version = 2.0.0, solver = HiGHS.


┌ Warning: Model period emission limits (ModelPeriodEmissionLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/p49Bn/src/scenario_calculation.jl:6112


2024-12-Nov 19:47:58.606 Finished modeling scenario.


2024-11-12 19:47:58,828 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2024-11-12 19:47:58,848 - INFO - EnergyProduction model run successfully completed
2024-11-12 19:47:58,849 - INFO - Running Energy (Fugitive Emissions)
2024-11-12 19:47:58,898 - INFO - Fugitive Emissions from Energy model run successfully completed
2024-11-12 19:47:58,898 - INFO - Appending Socioeconomic outputs
2024-11-12 19:47:58,908 - INFO - Socioeconomic outputs successfully appended.
2024-11-12 19:47:58,912 - INFO - Model run for primary_id = 0 successfully completed in 97.4 seconds (n_tries = 1).
2024-11-12 19:47:58,919 - INFO - Trying run primary_id = 1001 in region croatia
2024-11-12 19:47:58,920 - INFO - Running AFOLU model
2024-11-12 19:47:59,107 - INFO - AFOLU model run successfully completed
2024-11-12 19:47:59,108 - INFO - Running CircularEconomy model
2024-11-12 19:47:59,165 - INFO - CircularEconomy model run successfully completed
2024-11-12 19:47:59,166 - INFO - Running IPPU mod

2024-12-Nov 19:48:01.520 Started modeling scenario. NEMO version = 2.0.0, solver = HiGHS.


┌ Warning: Model period emission limits (ModelPeriodEmissionLimit parameter) are not enforced when modeling selected years.
└ @ NemoMod ~/.julia/packages/NemoMod/p49Bn/src/scenario_calculation.jl:6112


2024-12-Nov 19:49:00.326 Finished modeling scenario.


2024-11-12 19:49:00,586 - INFO - NemoMod ran successfully with the following status: OPTIMAL
2024-11-12 19:49:00,608 - INFO - EnergyProduction model run successfully completed
2024-11-12 19:49:00,609 - INFO - Running Energy (Fugitive Emissions)
2024-11-12 19:49:00,664 - INFO - Fugitive Emissions from Energy model run successfully completed
2024-11-12 19:49:00,665 - INFO - Appending Socioeconomic outputs
2024-11-12 19:49:00,675 - INFO - Socioeconomic outputs successfully appended.
2024-11-12 19:49:00,679 - INFO - Model run for primary_id = 1001 successfully completed in 61.76 seconds (n_tries = 1).
2024-11-12 19:49:00,703 - INFO - 
***	 REGION croatia COMPLETE	***

2024-11-12 19:49:00,785 - INFO - Table MODEL_OUTPUT successfully written to /home/tony-ubuntu/anaconda3/envs/ssp_env/lib/python3.11/site-packages/sisepuede/out/sisepuede_run_2024-11-12T19;45;23.670544/sisepuede_run_2024-11-12T19;45;23.670544_output_database/MODEL_OUTPUT.csv.
2024-11-12 19:49:00,789 - INFO - Table ATTRIBUTE_PR

{'croatia': [0, 1001]}

In [69]:
# INPUTS_ESTRESADOS_FILE_PATH = build_path([INPUTS_ESTRESADOS_PATH, f"sim_input_{id_experimento}.csv"])
# OUTPUTS_ESTRESADOS_FILE_PATH = build_path([OUTPUTS_ESTRESADOS_PATH, f"sim_output_{id_experimento}.csv"])


df_out = ssp.read_output(None)
# df_out.to_csv(OUTPUTS_ESTRESADOS_FILE_PATH, index=False)
# df_estresado[campos_estresar].to_csv(INPUTS_ESTRESADOS_FILE_PATH, index=False)

# print_elapsed_time(start_time)

In [70]:
df_out.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,croatia,0,16534.283819,15353.968471,23641.309795,18112.738265,18637.129748,24004.437783,18690.280291,...,115451.790769,9.795368e+05,16212.659335,919.674897,36.817737,30035.563529,0.000000,0.000000,8.836262e+05,200983.121397
1,0,croatia,1,16838.599816,15636.560585,24076.431681,18446.105956,18980.148943,24446.243096,19034.277729,...,117748.781628,1.024344e+06,16236.593070,1648.013009,1464.611003,37724.224890,4020.095501,556.957295,1.195545e+06,200448.604883
2,0,croatia,2,17131.792861,15908.823770,24495.649573,18767.288835,19310.630558,24871.900129,19365.701833,...,114802.836660,1.055492e+06,16247.989427,1648.013009,1464.611003,37699.078862,4020.095501,556.957295,1.051240e+06,220615.254998
3,0,croatia,3,17414.141193,16171.016401,24899.361307,19076.591705,19628.888219,25281.812832,19684.867121,...,137386.367646,1.072786e+06,16223.003802,1648.013009,1464.611003,39123.780050,4020.095501,556.957295,9.733595e+05,215992.829602
4,0,croatia,4,17685.916585,16423.390852,25287.955473,19374.312283,19935.228263,25676.375763,19992.080805,...,130801.638001,1.089419e+06,16165.211300,1648.013009,1464.611003,49441.921019,4020.095501,556.957295,1.051316e+06,223080.782882


In [71]:
columns_with_na = df_out.columns[df_out.isna().any()].tolist()
print(columns_with_na)

[]


In [72]:
columns_all_zeros = df_out.columns[(df_out == 0).all()].tolist()

In [73]:
columns_all_zeros

['area_lndu_conversion_from_croplands_to_forests_mangroves',
 'area_lndu_conversion_from_croplands_to_forests_primary',
 'area_lndu_conversion_from_forests_mangroves',
 'area_lndu_conversion_from_forests_mangroves_to_croplands',
 'area_lndu_conversion_from_forests_mangroves_to_forests_mangroves',
 'area_lndu_conversion_from_forests_mangroves_to_forests_primary',
 'area_lndu_conversion_from_forests_mangroves_to_forests_secondary',
 'area_lndu_conversion_from_forests_mangroves_to_grasslands',
 'area_lndu_conversion_from_forests_mangroves_to_other',
 'area_lndu_conversion_from_forests_mangroves_to_settlements',
 'area_lndu_conversion_from_forests_mangroves_to_wetlands',
 'area_lndu_conversion_from_forests_primary_to_croplands',
 'area_lndu_conversion_from_forests_primary_to_forests_mangroves',
 'area_lndu_conversion_from_forests_primary_to_forests_secondary',
 'area_lndu_conversion_from_forests_primary_to_other',
 'area_lndu_conversion_from_forests_primary_to_settlements',
 'area_lndu_con

In [74]:
df_out.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yield_agrc_fruits_tonne,yield_agrc_herbs_and_other_perennial_crops_tonne,yield_agrc_nuts_tonne,yield_agrc_other_annual_tonne,yield_agrc_other_woody_perennial_tonne,yield_agrc_pulses_tonne,yield_agrc_rice_tonne,yield_agrc_sugar_cane_tonne,yield_agrc_tubers_tonne,yield_agrc_vegetables_and_vines_tonne
0,0,croatia,0,16534.283819,15353.968471,23641.309795,18112.738265,18637.129748,24004.437783,18690.280291,...,115451.790769,9.795368e+05,16212.659335,919.674897,36.817737,30035.563529,0.000000,0.000000,8.836262e+05,200983.121397
1,0,croatia,1,16838.599816,15636.560585,24076.431681,18446.105956,18980.148943,24446.243096,19034.277729,...,117748.781628,1.024344e+06,16236.593070,1648.013009,1464.611003,37724.224890,4020.095501,556.957295,1.195545e+06,200448.604883
2,0,croatia,2,17131.792861,15908.823770,24495.649573,18767.288835,19310.630558,24871.900129,19365.701833,...,114802.836660,1.055492e+06,16247.989427,1648.013009,1464.611003,37699.078862,4020.095501,556.957295,1.051240e+06,220615.254998
3,0,croatia,3,17414.141193,16171.016401,24899.361307,19076.591705,19628.888219,25281.812832,19684.867121,...,137386.367646,1.072786e+06,16223.003802,1648.013009,1464.611003,39123.780050,4020.095501,556.957295,9.733595e+05,215992.829602
4,0,croatia,4,17685.916585,16423.390852,25287.955473,19374.312283,19935.228263,25676.375763,19992.080805,...,130801.638001,1.089419e+06,16165.211300,1648.013009,1464.611003,49441.921019,4020.095501,556.957295,1.051316e+06,223080.782882
